<h1> Captsone Project - Car Accident Severity </h1>

<h2> Introduction/Business Problem </h2>

Montreal, the 2nd most populated city in Canada has more than 4 millions population with a slow growth of 0.73% average every year. It also has a population density of 4,517 per KM², covering 431.5 km² of land. Montreal is also known as the city with 2nd largest economy in Canada by having a variety of businesses implementing themselves out there. As a place with growing work possibilities, many people consider moving there. On the other hand, you can expect traffic with that much people and the economical activity. Along traffic, car accidents is also a frequent thing in the french city and severity of the events are ranging from minor damage to casualties. 

If an event had to happen, We would like to know case study will use machine learning algorithms to predict future event severity. This process targets the city's civil engineers and any road responsible in case those incidents are street related. This could also interest the ones who would like to open a car body shop to repair vehicules related to these events.

<h2> Data Description </h2>

Data :<br>

All accidents records in Montreal.<br>
- Data pulled from the city of Montreal website in the format of a CSV file.
- Severity(From minor material damage(1), material damage only(2), light(3) and severe injuries(4) and casualties(5))
- Amount of type of vehicules(cars, trucks, bicycle, motocycle and etc.)
- Type of event(Object on which another collided on)
- Number of injures, casualties
- Locations (Longitude and Latitude)    

How : 

Several platform and techniques will be used during this report.
- Python as the interpreter language. As R, extensively used in Data analytic field. Useful for the diversity of libraries.
- Geocoders to convert address into coordinates value.
- Pandas for dataframe manipulation.
- Folium for map vizualization with our point of interest(Neighbordhoods and venues).
- Machine Learning Clustering algorithm such as K Means, Decision Trees, Support Vector Machine and Logistic Regression.


In [2]:
url = "http://donnees.ville.montreal.qc.ca/dataset/cd722e22-376b-4b89-9bc2-7c7ab317ef6b/resource/05deae93-d9fc-4acb-9779-e0942b5e962f/download/collisions_routieres.csv"


In [7]:
%pip install -q geocoder geopy folium bs4 pandas lxml html5lib sklearn matplotlib OSMPythonTools
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
import geocoder
import numpy as np
import requests
import io
import matplotlib.pyplot as plt
import unidecode
import branca.colormap as cm
from io import StringIO
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from folium import plugins
from folium.plugins import HeatMap

Note: you may need to restart the kernel to use updated packages.


In [24]:
s = requests.get(url).content
carDF = pd.read_csv(io.StringIO(s.decode('utf-8')))
#carDF.head()

In [27]:
carDFLimited = pd.DataFrame([carDF.NO_SEQ_COLL,carDF.JR_SEMN_ACCDN,carDF.DT_ACCDN,carDF.AN,carDF.HEURE_ACCDN,carDF.GRAVITE,carDF.LOC_LAT,carDF.LOC_LONG]).transpose()

In [36]:
#carDF.isna()
carDFLimited.replace({'GRAVITE': {"Dommages matériels inférieurs au seuil de rapportage": 1, "Dommages matériels seulement" : 2, "Léger" : 3, "Grave" : 4, "Mortel" : 5}},inplace=True)
carDFLimited.head(100)
#carDF['DATE'] = pd.to_datetime(montrealcrime['DATE'])

,NO_SEQ_COLL,JR_SEMN_ACCDN,DT_ACCDN,AN,HEURE_ACCDN,GRAVITE,LOC_LAT,LOC_LONG
0,SPVM _ 2012 _ 1,ME,2012/02/01,2012,Non précisé,1,45.4555,-73.8616
1,SPVM _ 2012 _ 2,SA,2012/06/02,2012,15:00:00-15:59:00,1,45.4877,-73.716
2,SPVM _ 2012 _ 3,JE,2012/06/28,2012,08:00:00-08:59:00,1,45.5143,-73.6828
3,SPVM _ 2012 _ 4,ME,2012/07/11,2012,09:00:00-09:59:00,1,45.4922,-73.5765
4,SPVM _ 2012 _ 5,LU,2012/01/02,2012,Non précisé,1,45.4893,-73.8501
...,...,...,...,...,...,...,...,...
95,SPVM _ 2012 _ 96,ME,2012/02/22,2012,18:00:00-18:59:00,1,45.504,-73.8454
96,SPVM _ 2012 _ 97,VE,2012/02/24,2012,20:00:00-20:59:00,3,45.4907,-73.8536
97,SPVM _ 2012 _ 98,VE,2012/02/24,2012,16:00:00-16:59:00,1,45.4906,-73.8715
98,SPVM _ 2012 _ 99,VE,2012/02/24,2012,16:00:00-16:59:00,1,45.5058,-73.8221


In [ ]:
map = folium.Map(location=Montreal, zoom_start=11)
plugins.ScrollZoomToggler().add_to(map)
for borough,Type,lat,long in zip(carDFLimited['GRAVITE'],carDFLimited['Lat'],carDFLimited['Long']):
    label = folium.Popup(borough, parse_html=True)
    color='blue'
    if Type == 'B':
        color='blue'
    else:
        color='green'
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
map